# 4. Praca z dużymi plikami tekstowymi: komendy sed i grep. Uprawnienia pliku i interaktywność.

Bardzo często podczas pracy z plikami zawierającymi duże ilości danych (wielokolumnowe i wielowierszowe pliki) jest potrzeba przefiltrowania takiego pliku, wyszukania w nim pewnej frazy lub zamiany pewnych wartości na inne. W tej części zaprezentowane zostaną spospoby jak takie czynności wykonać w `bash`-u. Jednak na początek, warto poznać inną komendę dzięki której można nadać danemu plikowi __uprawnienia__. Wiąże się to z innym sposobem uruchamiania skryptów.

## 4.1 Uprawnienia plików - `chmod`

Komenda `chmod` służy do zmiany/nadania uprawnień dostępu do pliku w systemach UNIX (do tej rodizny należy Linux). Sposób użycia:

``` bash
chmod [opcje] <uprawnienia> <plik>
```
Na początek sprawdźmy jakie uprawnienia mają nasze pliki:

In [2]:
ls -l

razem 1264
-rw-r--r-- 1 krzkot krzkot  32737 mar  9 17:04 1_komendy.ipynb
-rw-r--r-- 1 krzkot krzkot  16164 mar 17 21:31 2_pierwszy_skrypt.ipynb
-rw-r--r-- 1 krzkot krzkot  18413 mar 26 13:55 3_powloka_i_zmienne.ipynb
-rw-r--r-- 1 krzkot krzkot   3482 mar 26 15:04 4_duze_pliki_tekstowe.ipynb
-rw-rw-r-- 1 krzkot krzkot    210 mar 17 20:36 baza.bash
drwxr-xr-x 3 krzkot krzkot   4096 mar 19 16:14 exercises
-rw-r--r-- 1 krzkot krzkot 824264 mar  5 10:36 TIC102090493-s2-121s_lc3_fppt.dat
-rw-r--r-- 1 krzkot krzkot 383940 mar 17 20:39 TIC12974182-s2-121s_lc3_fppt.dat


Uprawniania naszych plików wypisane są w pierwszej kolumnie, oznaczone przez litery `rwx`. Oznaczają one po kolei:
- `r` -> oznacza, że plik można odczytać (__r__ead)
- `w` -> oznacza, że do pliku można wpisać (__w__rite) jakąś wartość
- `x` -> oznacza, że plik można uruchomić (e __x__ecute) (jak np. skrypt)

Żeby zrozumieć ten zapis typu: `-rw-r--r--`, trzeba wiedzieć, że w systemie Linux (w Windowsie też) każdy plik lub folder może mieć swojego właściciela (na jednym komputerze może być wiele użytkowników). Dodatkowo użytkownicy mogą być zebrani w grupy i danej grupie użytkowników można nadać uprawnienia naraz. 

Po wykonaniu komendy `ls -l` zobaczymy jakie uprawnienia mają właściciel, grupa oraz inni użytkownicy. Dla przykładu weźmy `-rw-r--r--`:
 - pierwsze 4 znaki odnoszą się do właściciela pliku, czyli tutaj będzie mógł odczytać i zapisać do pliku
 - kolejne 3 znaki odnoszą się do grupy, czyli tutaj będzie mógł tylko odczytać plik
 - ostatnie 3 znaki odnoszą się do innych użytkowników, czyli tutaj będzie mógł tylko odczytać plik
 - litera `d` na początku oznacza, że jest to folder (__d__irectory)


Aby zmienić uprawnienia należy użyć komendy `chmod` i wyspecyfikować jakie uprawnienia chcemy __dodać/odebrać/zmienić__, za pomocą odpowienio __+/-/=__ i komu - __użytkownik/grupa/inni__, za pomocą __u/g/o__. Dla przykładu, stworzę pusty plik i nadam mu uprawnienia odczytu, zapisu i uruchamiania dla wszystkich użytkowników (u oraz o). Domyślnie utworzy się z uprawnieniami `rw-r--r-- `.

In [28]:
touch pustyplik; ls -l pustyplik

-rw-r--r-- 1 krzkot krzkot 0 mar 26 16:34 pustyplik


In [29]:
chmod ou-w pustyplik; ls -l pustyplik            # usuwam uprawnienia w dla użytkowników

-r--r--r-- 1 krzkot krzkot 0 mar 26 16:34 pustyplik


In [31]:
echo "text" > pustyplik                          # przy próbie wpisania czegoś do pliku występuje błąd

bash: pustyplik: Brak dostępu


: 1

In [33]:
chmod uo=rwx  pustyplik; ls -l pustyplik         # nadaję uprawnienia rwx dla użytkowników

-rwxr--rwx 1 krzkot krzkot 0 mar 26 16:34 pustyplik


I w końcu dodanie uprawnień:

In [34]:
chmod g+x pustyplik; ls -l pustyplik             # nadaje uprawnienia x dla grupy

-rwxr-xrwx 1 krzkot krzkot 0 mar 26 16:34 pustyplik


## 4.2 Inny sposób uruchamiania skryptu. 

Można naszemu skryptowi nadać uprawnienia wykonywania (u+x), aby uruchomić go za pomocą:
`./nazwa_skryptu.bash`<br><br>
__[UWAGA]__ Aby ten sposób działał, skrypt musi się rozpoczynać od preambuły `#!/bin/bash`

In [19]:
ls -l baza.bash

-rw-rw-r-- 1 krzkot krzkot 210 mar 17 20:36 baza.bash


In [21]:
 chmod u+x baza.bash; ls -l baza.bash

-rwxrw-r-- 1 krzkot krzkot 210 mar 17 20:36 baza.bash


In [24]:
./baza.bash TIC102090493-s2-121s_lc3_fppt.dat    # uruchamiam skrypt, po nadaniu uprawnień wykonywania

27.4044052


## 4.3 Praca z plikiem tekstowym: `sed` (stream editor)

Komenda `sed` jest bardzo użyteczną komendą dzięki, której można w rózny sposób manipulować plikiem. Tak naprawdę jest to dość zaawansowany program, który sam w sobie może być językiem programowania. Służy on do przetwarzania tekstów. Operacje zazwyczaj dobrze jest "opakować" apostrofami 'operacje' - wtedy rozpoznawane są w całości jako ciąg znaków. Warto użyć cudzysłowia w przypadku łączenia ze zmiennymi - będą miały podstawioną wartość.
Uruchomienie:
```bash
sed [opcje] 'operacje' plik
```
Najprostsze wykonanie z użyciem pliku `equations.dat`:

In [47]:
sed -n '5p' equations.dat       # wyświetla 5 linię z pliku

477.15 / 217.23


W ogólności `sed` ma następujące opcje:
 - -n : wypisane zostaną tylko te linie na które zadziałano za pomocą komepndy p lub s (wyjaśnienie niżej)
 - -e : następnym argumentem będzie polecenie edycyjne (w powyższym przykładzie dla formalności można było wpisać:
 
 `sed -ne '5p' equations.dat`
 - -f : następnym argumentem jest plik z poleceniami dla `sed`-a

W poleceniach możemy używać następujących komend:
- p : (print) wypisanie
- s : (substitute) zamiana
- d : (delete) usunięcie
- g : (global) zadziałanie na całej linii, nie tylko na pierwszym wystąpieniu (w połączeniu z s)

Kilka przykładów:

In [51]:
sed -ne '3,7p' equations.dat    # wypisanie linii od 3 do 7

805.51 + 981.97
296.5 / 132.77
477.15 / 217.23
149.96 - 418.28
707.83 / 241.16


In [58]:
sed -e '1,9997d' equations.dat   # nie wypisze linii od 1 do 9997

320.97 - 672.71
162.48 + 907.06
260.32 + 804.84


Za pomocą `sed`-a można wyszukiwać wzorce w pliku i wypisywać je lub zamieniać. Wzorce (tzw. __wyrażenia regularne__) definiuje się za pomocą następujących znaków:

| Znak  |     Znaczenie     | | |* * * * * *| | | Wyrażenie  |     Znaczenie     |
| :---: | :---------------: |-|-|-|-|-| :---: | :---------------: |
|   ^   | Początek wiersza  | | |* * * * * *| | | /./  | Wiersz zawierający co najmniej jeden znak
|   $   |   Koniec wiersza  | | |* * * * * *| | | /../ | Wiersz zawierający co najmniej dwa znaki
|   .   |    Dowolny znak   | | |* * * * * *| | | /[abc]/ | Wiersz zawierający a, b lub c
|   *   |0 lub więcej powtórzeń poprzedniego znaku| | |* * * * * *| | | /^#/ | Wiersz zaczynający się od #/
|  [ ]  | Znak pomiędzy “[“ oraz “]” | | |* * * * * *| | | }^/ | Wiersz kończący się “}”

Jeśli potrzebne jest wyszukanie znaku, który jest __zastrzeżony__ w powyższej tabeli, należy przed takim znakiem umieścić backslash `\`. Komendy można również łączyć za pomocą potoku:

In [75]:
head equations.dat | sed -e 's/\./kropka/g'  # zamieni znak '.' na wyraz 'kropka' w całym strumieniu

724kropka96 / 1kropka37
782kropka65 - 848kropka9
805kropka51 + 981kropka97
296kropka5 / 132kropka77
477kropka15 / 217kropka23
149kropka96 - 418kropka28
707kropka83 / 241kropka16
750kropka53 - 329kropka8
401kropka87 / 319kropka88
717kropka33 - 189kropka74


Poniższa komenda zamieni wyrażenie które zawiera słowo `kropka` oraz 3 znaki przed i 2 znaki po nim na słowo `pierwsza liczba`

In [87]:
head equations.dat | sed -e 's/\./kropka/g' | sed -e 's/...kropka../pierwsza_liczba /'

pierwsza_liczba  / 1kropka37
pierwsza_liczba  - 848kropka9
pierwsza_liczba  + 981kropka97
pierwsza_liczba / 132kropka77
pierwsza_liczba  / 217kropka23
pierwsza_liczba  - 418kropka28
pierwsza_liczba  / 241kropka16
pierwsza_liczba  - 329kropka8
pierwsza_liczba  / 319kropka88
pierwsza_liczba  - 189kropka74


Poniższe, zamieni wyrażenie `liczba` na `cyfra` tylko dla 5 pierwszych linii

In [91]:
head equations.dat | sed -e 's/\./kropka/g' | sed -e 's/...kropka../pierwsza_liczba /' > zmiana.dat

In [92]:
cat zmiana.dat

pierwsza_liczba  / 1kropka37
pierwsza_liczba  - 848kropka9
pierwsza_liczba  + 981kropka97
pierwsza_liczba / 132kropka77
pierwsza_liczba  / 217kropka23
pierwsza_liczba  - 418kropka28
pierwsza_liczba  / 241kropka16
pierwsza_liczba  - 329kropka8
pierwsza_liczba  / 319kropka88
pierwsza_liczba  - 189kropka74


In [94]:
sed -e '1,5s/liczba/cyfra/' zmiana.dat      # Ta komenda zmieni słowo liczba na cyfra dla pierwszych 5 linii

pierwsza_cyfra  / 1kropka37
pierwsza_cyfra  - 848kropka9
pierwsza_cyfra  + 981kropka97
pierwsza_cyfra / 132kropka77
pierwsza_cyfra  / 217kropka23
pierwsza_liczba  - 418kropka28
pierwsza_liczba  / 241kropka16
pierwsza_liczba  - 329kropka8
pierwsza_liczba  / 319kropka88
pierwsza_liczba  - 189kropka74


## 4.4 Praca z plikiem tekstowym: `grep`

`grep` pozwala na wyszukiwanie danej frazy/wzorca w pliku. Wzorzec dobrze jest "opakować" apotstrofami `'wzorzec'` - wtedy rozpoznawany jest w całości jako ciąg znaków. W przypadku cudzysłowia następujący wzorzec: `"$wyraz"` zostanie rozpoznany jako wartość zmiennej `wyraz`.<br>

__[UWAGA]__ `grep` jest czuły na wielkość liter<br>

Składnia jest następująca:
```bash
grep [-opcje] wzorzec [plik/ścieżka]
```

Użyteczne opcje do użycia w komendzie `grep`:
- -i ignoruje wielkość liter
- -r wyszukuje również w sub katalogach ścieżki
- -l pokazuje nazwy plików, w których znajduje się pasujący wzorzec
- -n pokazuje linię, w której występuje szukany wzorzec
- -v pokazuje linie, które nie zawierają wzorca
- -e informuje, że następny argument, jest wzorcem

Przykłady:

In [123]:
grep -n '264' equations.dat    # Wyszukuje ciąg znaków: 264 w pliku i pokazuje w której linii się znajduje

19:264.97 - 821.85
521:264.25 + 296.13
1697:735.52 + 264.59
1951:264.76 + 414.02
2496:209.06 - 264.16
3051:264.78 - 409.38
3432:264.78 - 356.58
3472:531.44 / 264.18
3852:21.11 + 264.36
4462:358.78 - 264.2
4971:362.49 - 264.34
5341:407.73 - 264.95
5597:495.27 - 264.44
5745:264.09 - 806.29
6270:264.45 / 540.79
8370:264.15 / 984.58
8895:754.05 + 264.56


Można również stosować __wyrażenia regularne__ podobnie jak w komendzie `sed`:

In [121]:
grep -v '[0-5]' equations.dat    # wypisuje te linie które nie zawierają cyfr od 0 do 5

86.96 - 77.89
989.66 + 699.96
879.79 + 776.89


## 4.5 Interaktywność: `read`

W `bash`-u można również pisać skrypty, które będą w pewnym stopniu interaktywne. Służy do tego komenda `read`. W takim skrypcie należy umieścić linię:
```bash
read jakaszmienna
```
Gdzie `jakaszmienna` to downolna nazwa zmiennej, do której wpisane będzie to co użytkownik wpisze. Po wpisaniu należy wcisnąć klaiwsz `enter`.<br>

__Przykład__<br>
Stworzę skrypt o nazwie witaj.bash o takiej zawartości:
```bash
#!bin/bash
echo "Podaj swoje imię: "
read imie
echo "Witaj, "$imie
```

In [134]:
chmod u+x witaj.bash; ls -l witaj.bash

-rwxr--r-- 1 krzkot krzkot 77 mar 27 16:04 witaj.bash


Uruchomienie go w terminalu da nastepujący efekt (komenda exit na końcu jest nie związana ze skryptem, wynika ona z nagrywania gif-a):

![SegmentLocal](witaj.gif "segment")